# INSTALL & IMPORTS

In [12]:
#pip install selenium -q
#pip install requests -q
# pip install bs4 -q 

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# TESTING

In [42]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [81]:
page_source = driver.page_source
profile = BeautifulSoup(page_source, 'html.parser')
name = profile.find("span", class_='encounters-story-profile__name').text
age = profile.find("span", class_= "encounters-story-profile__age").text
about = profile.find("ul", class_= "encounters-story-about__badges").text if profile.find("ul", class_= "encounters-story-about__badges") else ""
if about:
    about = re.split('(?=[A-Z])', about)

location = profile.find("span", class_="header-2 text-color-black").text
more_info = profile.find("p", class_="encounters-story-about__text").text
job = profile.find("p", class_="encounters-story-profile__occupation").text
education = profile.find("p", class_="encounters-story-profile__education").text


print(name, age, about, location, more_info, job, education)

Joan , 30 ['174 cm', 'Active', 'Aries', 'Graduate degree', 'Socially', 'Never', 'Man', 'Something casual', 'Open to kids', 'Agnostic'] Barcelona Buscant còmplice per tast de cerveses

That's what she said Advocat Universitat Pompeu Fabra 2015


In [83]:
education

'Universitat Pompeu Fabra 2015'

In [54]:
# Esperar a que el botón esté presente y sea clickeable
try:
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
    )
    button.click()
    print("Botón clickeado con éxito")
except Exception as e:
    print("No se pudo clickear el botón:", e)


Botón clickeado con éxito


# FUNCTION TO TAKE ALL INFO

In [221]:
# Inicializar WebDriver
driver = webdriver.Chrome()

# Abrir la página de Bumble
driver.get("https://fr1.bumble.com/app")

In [253]:
def extract_profile_info():

    profiles = []
    prevJob = ""
    prevStories = ""
    prevEducation = ""
    
    for i in range(2):
        
        # Obtener el contenido de la página actual
        page_source = driver.page_source
    
        # Analizar el contenido de la página con BeautifulSoup
        profile = BeautifulSoup(page_source, 'html.parser')
    
    
        # Realizar un diccionario para que la información se vaya guardando.
        profiles_dict = {}
    
        # Esperar 15 segundos antes de extraer la información
        time.sleep(5)
    
        # Extraer información del perfil
        name = profile.find("span", class_='encounters-story-profile__name')
        profiles_dict["name"] = name.text if name else None
        
        age = profile.find("span", class_= "encounters-story-profile__age")
        profiles_dict["age"] = age.text.strip(",")  if age else None

        
        location = profile.find("span", class_="header-2 text-color-black")
        profiles_dict["location"] = location.text if location else ""
    

        story_about = profile.find("p", class_="encounters-story-about__text")
        profiles_dict["story_about"] = story_about.text if story_about and story_about.text != prevStories else None
        if story_about:
            prevStories = story_about.text

        
        job = profile.find("p", class_="encounters-story-profile__occupation")
        profiles_dict["job"] = job.text if job and job.text != prevJob else None
        if job:
            prevJob = job.text
    
        education = profile.find("p", class_="encounters-story-profile__education")
        profiles_dict["education"] = education.text if education and education.text != prevEducation else None
        if education:
            prevEducation = education.text

        # Encuentra todos los elementos div con la clase "pill"
        about = profile.find_all("div", class_="pill")  

        #categories_dict = {}
        
        for pill in about:
            image_tag = pill.find("img")
            
            if image_tag is not None:
                image_url = image_tag['src']
                text = pill.get_text(strip=True)
                category = image_url.split('_')[-1].split('.')[0]
                if category is not None and category not in profiles_dict:
                    profiles_dict[category] = text
            
            else:
                category = None
                text = None
                
    

        profiles.append(profiles_dict)
    
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.encounters-action__icon [data-qa-icon-name="floating-action-no"]'))
            )
            time.sleep(5)  # Añadir un temporizador adicional de 5 segundos
            button.click()
            print("Botón clickeado con éxito")
        except Exception as e:
            print("No se pudo clickear el botón:", e)
            

    return profiles

In [254]:
profile_info = extract_profile_info() 

# Make a DataFrame from a dict. 
df = pd.DataFrame(profile_info)

# Print DataFrame
df

Botón clickeado con éxito
Botón clickeado con éxito


,name,age,location,story_about,job,education,heightv2,exercisev2,starSignv2,educationv2,drinkingv2,smokingv2,genderv2,intentionsv2,familyPlansv2,religionv2
0,Hailey,22,Barcelona,"Hello, hope everyone is doing good. I not real...",None,None,170 cm,Active,Taurus,In college,Socially,Never,Woman,Relationship,Open to kids,Other
1,Marcos,19,Barcelona,None,None,None,170 cm,Active,Taurus,In college,Socially,Never,Man,Something casual,Open to kids,Other


In [214]:
df.to_csv('df_prueba.csv', index=False)

In [252]:
df.head()

,name,age,heightv2,exercisev2,starSignv2,educationv2,drinkingv2,smokingv2,genderv2,intentionsv2,familyPlansv2,religionv2,Politicsv2,location,story_about,job,education
0,Luis,34,160 cm,Active,Aries,Undergraduate degree,Rarely,Never,Man,Don’t know yet,Not sure yet,Agnostic,Apolitical,Barcelona,Empatía,None,None
1,Ahmed,31,186 cm,Sometimes,Scorpio,Graduate degree,Socially,Socially,Man,Relationship,Not sure yet,Agnostic,Apolitical,Barcelona,None,None,None
